# Implementieren einer lokalen Suchmaschine

## Einführung
Im 2. Assignment werden wir einen lokalen Such-Algorithmus entwickeln, der die Informationen über die Volleyballspielerinnen des österreichischen Nationalteams effizient durchsuchen kann. Dazu werden wir zunächst die Daten des Kaders, die im 1. Assignment gesammelt wurden, verwenden.

## Arbeitsaufteilung
**Ecker Annina**: Implementierung des Inverted Index

**Cesar Laura**: TD-IDF Scoring + User Interface

**Dilly Julian**: Implementierung der lokalen Suchmaschine

### 1. Installation und Importieren der benötigten Libraries

In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
from collections import defaultdict, Counter
import re
import nltk
import math
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /Users/laura/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/laura/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 2. Importieren der bereinigten Spielerinnendaten als CSV

Die CSV-Daten sind bereits bereinigt. In 3.2 werden die Daten nochmals bereinigt, indem strings auf lowercase() gebracht und Sonderzeichen entfernt werden.

In [3]:
spielerinnen_csv = pd.read_csv('./Spielerinnendaten.csv')
spielerinnen_csv

,Dressnummer,Name,Position,Nationalitaet,Groesse
0,1,Nicole Leonie HOLZINGER,Aufspiel,AUT,175 cm
1,2,Carmen RAAB,Diagonal,AUT,179 cm
2,3,Tamina HUBER,Libero,AUT,173 cm
3,10,Julia TRUNNER,Diagonal,AUT,185 cm
4,11,Monika CHRTIANSKA,Außenangriff,AUT,183 cm
5,13,Lina HINTEREGGER,Außenangriff,AUT,180 cm
6,14,Kora Marina SCHABERL,Außenangriff,AUT,183 cm
7,15,Anna OBERHAUSER,Libero,AUT,166 cm
8,17,Dana SCHMIT,Aufspiel,AUT,175 cm
9,18,Nina NESIMOVIC,Mittelblock,AUT,188 cm


### 3. Implementierung des Inverted Index
#### 3.1 Inhalt des CSVs in Token splitten

In [4]:
# Es werden alle Zeilen und Spalten durchlaufen, um die Inhalte in Token aufzuteilen
for index, row in spielerinnen_csv.iterrows():
    for col in spielerinnen_csv.columns:
        cell_value = str(row[col])  # Konvertierung des Zelleninhalt in einen String, falls er kein Text ist
        tokens = word_tokenize(cell_value)
        print(f"Zeile {index}, Spalte {col}: {tokens}")

# Letzer Token außerhalb der Schleife wird hier zusätzlich ausgegeben
display(tokens)

Zeile 0, Spalte Dressnummer: ['1']
Zeile 0, Spalte Name: ['Nicole', 'Leonie', 'HOLZINGER']
Zeile 0, Spalte Position: ['Aufspiel']
Zeile 0, Spalte Nationalitaet: ['AUT']
Zeile 0, Spalte Groesse: ['175', 'cm']
Zeile 1, Spalte Dressnummer: ['2']
Zeile 1, Spalte Name: ['Carmen', 'RAAB']
Zeile 1, Spalte Position: ['Diagonal']
Zeile 1, Spalte Nationalitaet: ['AUT']
Zeile 1, Spalte Groesse: ['179', 'cm']
Zeile 2, Spalte Dressnummer: ['3']
Zeile 2, Spalte Name: ['Tamina', 'HUBER']
Zeile 2, Spalte Position: ['Libero']
Zeile 2, Spalte Nationalitaet: ['AUT']
Zeile 2, Spalte Groesse: ['173', 'cm']
Zeile 3, Spalte Dressnummer: ['10']
Zeile 3, Spalte Name: ['Julia', 'TRUNNER']
Zeile 3, Spalte Position: ['Diagonal']
Zeile 3, Spalte Nationalitaet: ['AUT']
Zeile 3, Spalte Groesse: ['185', 'cm']
Zeile 4, Spalte Dressnummer: ['11']
Zeile 4, Spalte Name: ['Monika', 'CHRTIANSKA']
Zeile 4, Spalte Position: ['Außenangriff']
Zeile 4, Spalte Nationalitaet: ['AUT']
Zeile 4, Spalte Groesse: ['183', 'cm']
Zeile 5

['185', 'cm']

#### 3.2 Erstellen des Inverted Index
Jeder Token wird den entsprechenden Dokumenten zugeordnet, in denen es vorkommt, um eine strukturierte Darstellung der Daten zu erstellen.
In diesem Kontext bezieht sich "Dokumente" nicht auf physische Dokumente wie Papierseiten oder Dateien im herkömmlichen Sinne, sondern auf einzelne Textabschnitte oder Textblöcke, die durchsucht und indexiert werden. Dies könnten zum Beispiel Absätze, Sätze, Artikel, Tweets oder andere Textfragmente sein.


In [5]:
def clean_text(text):
    # Entfernt Sonderzeichen und macht alles in lowercase
    text = re.sub(r'\W+', ' ', text)
    return text.lower()

# Funktionsdefinition eines Inverted Index:
# In der Funktion build_inverted_index() wird jedem Text ein doc_id zugewiesen, das die Position des Textes in der Liste darstellt.
def build_inverted_index(docs):
    inverted_index = defaultdict(list)
    for doc_id, doc_fields in enumerate(docs):
        # Kombiniere alle Felder der Zeile zu einem einzigen String
        combined_fields = " ".join([str(field) for field in doc_fields])  # Konvertiere jedes Feld in einen String
        tokens = word_tokenize(clean_text(combined_fields))
        for token in tokens:
            if doc_id not in inverted_index[token]:
                inverted_index[str(token)].append(doc_id)  # Stelle sicher, dass der Token als String gespeichert wird
    return inverted_index

### 4. TD-IDF Scoring

```calculate_tf()``` calculates the term frequency (TF) for each token in a given document.
It tokenizes the document, counts the occurrences of each token, and then normalizes the count by dividing by the total number of tokens to get the term frequency.

```calculate_idf()``` calculates the inverse document frequency (IDF) for each token in the given set of documents using the previously caolculated inverted index.

```calculate_tfidf()``` calculates the TF-IDF score for each token in each document.
It first calculates IDF using calculate_idf, and then for each document, it computes the TF-IDF score by multiplying the term frequency (TF) with the corresponding IDF.
The results are stored in a DataFrame, where each document is represented as a row and each token as a column, with values being the TF-IDF scores.

```search()``` performs a search for a given search term in the TF-IDF DataFrame.
The documents are ranked based on their TF-IDF score for the search term and the ranked documents that have non-zero scores are printed out for validation.

In [6]:
def calculate_tf(doc):
    tokens = word_tokenize(clean_text(doc))
    total_tokens = len(tokens)
    tf = Counter(tokens)
    return {token: count / total_tokens for token, count in tf.items()}

def calculate_idf(documents, inverted_index):
    N = len(documents)
    idf = {}
    for token in inverted_index:
        df = len(inverted_index[token])
        idf[token] = math.log(N / (1 + df))  
    return idf

def calculate_tfidf(documents, inverted_index):
    idf = calculate_idf(documents, inverted_index)
    tfidf_documents = []
    
    for doc_fields in documents:
        combined_fields = " ".join([str(field) for field in doc_fields])
        tf = calculate_tf(combined_fields)
        tfidf = {token: tf[token] * idf.get(token, 0) for token in tf}
        tfidf_documents.append(tfidf)
    
    all_tokens = sorted(set(token for tfidf in tfidf_documents for token in tfidf))
    tfidf_df = pd.DataFrame(0.0, index=range(len(documents)), columns=all_tokens)
    
    for doc_id, tfidf in enumerate(tfidf_documents):
        for token, value in tfidf.items():
            tfidf_df.at[doc_id, token] = value
    
    return tfidf_df

def search(tfidf_df, search_term):
    search_term = search_term.lower()
    if search_term in tfidf_df.columns:
        ranked_docs = tfidf_df[search_term].sort_values(ascending=False)
        ranked_docs = ranked_docs[ranked_docs != 0]
        print(ranked_docs)
    else:
        print(f"'{search_term}' not found in player data.")

### Testing

In [7]:
player_documents = spielerinnen_csv[['Dressnummer', 'Name', 'Position', 'Nationalitaet', 'Groesse']].values.tolist()
player_inverted_index = build_inverted_index(player_documents)
player_tfidf_df = calculate_tfidf(player_documents, player_inverted_index)

search(player_tfidf_df, 'Mittelblock')

9     0.156945
11    0.156945
12    0.156945
13    0.156945
Name: mittelblock, dtype: float64


### 5. Implementierung der lokalen Suchmaschine

### 6. User Interface

Implementation with **Streamlit**:

Running the streamlit app:
1. navigate to ```Application``` directory
2. enter ```streamlit run app.py```

# Ressourcen & Source-Docs

## Inverted Index
1. [Implementieren eines Inverted Index](https://www.pingcap.com/article/step-by-step-guide-building-inverted-index-python/)

## User Interface
1. [Streamlit](https://streamlit.io/)

# Challenges bei der Implementierung


#### Cesar Laura - Calculating TD-IDF & User Interface
- Handling the results of 'AUT' as a search term. At first in the ```search``` function the line ```ranked_docs = ranked_docs[ranked_docs != 0]``` had > 0 as a condition which gave problems since due to the nature of the IDF formula ($math.log(N / (1 + df))$) the returned score of something that appears once in each document was negative.
- In the streamlit application it was a bit of a challenge handling the initial case of "" being the value of the search bar and adapting the functions to this edge case.


#### Dilly Julian
-
-

#### Ecker Annina
- 
- 